# Packages that need to be installed initially


In [183]:
# sc.install_pypi_package("pandas==0.25.1")
# sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [201]:
import pandas as pd
from pyspark.sql.functions import explode, col, split, regexp_replace, lit, concat
from pyspark.sql import functions as F

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
spark

# Disaster Declaration County/State

In [210]:
county = "Lee county"
state = "Florida"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Read JSON into DF

In [185]:
spark_disasters_df = spark.read.json("s3://real-estate-extracted-data/disasters/disaster_declarations-lehigh acres-fl.json")

spark_disasters_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- DisasterDeclarationsSummaries: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- declarationDate: string (nullable = true)
 |    |    |-- declarationTitle: string (nullable = true)
 |    |    |-- designatedArea: string (nullable = true)
 |    |    |-- disasterNumber: long (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- incidentBeginDate: string (nullable = true)
 |    |    |-- incidentType: string (nullable = true)
 |    |    |-- placeCode: string (nullable = true)
 |    |    |-- state: string (nullable = true)
 |-- metadata: struct (nullable = true)
 |    |-- DeprecationInformation: struct (nullable = true)
 |    |    |-- depApiMessage: string (nullable = true)
 |    |    |-- depDate: string (nullable = true)
 |    |    |-- depNewURL: string (nullable = true)
 |    |    |-- depWebMessage: string (nullable = true)
 |    |-- count: long (nullable = true)
 |    |-- entityname: string (nullable = true)
 |    |-- f

# Explode DisasterDeclarationsSummaries Array and Normalize DF

In [59]:
normalized_disaster_df = spark_disasters_df.select(explode(col('DisasterDeclarationsSummaries')).alias('data'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [93]:
cleaned_disasters_df = normalized_disaster_df.select("data.*")
cleaned_disasters_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------+--------------+--------------------+--------------------+---------------+---------+-----+
|     declarationDate|    declarationTitle|designatedArea|disasterNumber|                  id|   incidentBeginDate|   incidentType|placeCode|state|
+--------------------+--------------------+--------------+--------------+--------------------+--------------------+---------------+---------+-----+
|2017-09-05T19:12:...|      HURRICANE IRMA|  Lee (County)|          3385|60c3b85ba0ee349d7...|2017-09-04T21:00:...|      Hurricane|    99071|   FL|
|2019-08-30T10:35:...|    HURRICANE DORIAN|  Lee (County)|          3419|60c3b85ca0ee349d7...|2019-08-28T08:00:...|      Hurricane|    99071|   FL|
|2020-03-13T16:00:...|            COVID-19|  Lee (County)|          3432|60c3b85da0ee349d7...|2020-01-20T00:00:...|     Biological|    99071|   FL|
|2012-07-03T16:00:...|TROPICAL STORM DEBBY|  Lee (County)|          4068|60c3b863a0ee349d7...|2012-06-23T21:29:.

# Convert spark df to pandas 
### (Used after changes made to 'cleaned_disasters_df'. Only way to display data nicely.)

In [191]:
pandas_disasters_df = cleaned_disasters_df.toPandas()
pd.set_option('display.max_columns', None)
pandas_disasters_df.head(40)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

            declarationDate                  declarationTitle designatedArea  \
0  2017-09-05T19:12:00.000Z                    HURRICANE IRMA   Lee (County)   
1  2019-08-30T10:35:00.000Z                  HURRICANE DORIAN   Lee (County)   
2  2020-03-13T16:00:00.000Z                          COVID-19   Lee (County)   
3  2012-07-03T16:00:00.000Z              TROPICAL STORM DEBBY   Lee (County)   
4  2017-09-10T14:45:00.000Z                    HURRICANE IRMA   Lee (County)   
5  2020-03-25T12:45:00.000Z                 COVID-19 PANDEMIC   Lee (County)   
6  2017-04-22T01:08:00.000Z  LEHIGH ACRES (ANNA AVE. N.) FIRE   Lee (County)   
7  2021-07-04T21:25:00.000Z               TROPICAL STORM ELSA   Lee (County)   

   disasterNumber                        id         incidentBeginDate  \
0            3385  60c3b85ba0ee349d710a65ec  2017-09-04T21:00:00.000Z   
1            3419  60c3b85ca0ee349d710a7ee9  2019-08-28T08:00:00.000Z   
2            3432  60c3b85da0ee349d710a89fa  2020-01-20T00:0

# Create declaration year and month column from declarationDate

In [180]:
cleaned_disasters_df = cleaned_disasters_df.withColumn("declarationYear", split(F.col("declarationDate"), "-").getItem(0)) \
                    .withColumn("declarationMonth", split(F.col("declarationDate"), "-").getItem(1))
                                

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Create incident year and month from column incidentBeginDate

In [86]:
cleaned_disasters_df = cleaned_disasters_df.withColumn("yearIncidentBegan", split(F.col("incidentBeginDate"), "-").getItem(0)) \
                    .withColumn("monthIncidentBegan", split(F.col("incidentBeginDate"), "-").getItem(1))
                                

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Use regexp to replace '(county)' with "" into a new column

In [190]:
cleaned_disasters_df = cleaned_disasters_df.withColumn("countyName", regexp_replace('designatedArea', "\\(.+?\\)", ""))



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Create countyState column (concatinate)

In [212]:
cleaned_disasters_df.select(concat(col("countyName"),lit("County"),lit(", "),lit(state)).alias("countyState")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|        countyState|
+-------------------+
|Lee County, Florida|
|Lee County, Florida|
|Lee County, Florida|
|Lee County, Florida|
|Lee County, Florida|
|Lee County, Florida|
|Lee County, Florida|
|Lee County, Florida|
+-------------------+

# Groupby
### Incident type (count)

In [88]:
cleaned_disasters_df.groupby("incidentType").count().sort('count').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----+
|   incidentType|count|
+---------------+-----+
|           Fire|    1|
|     Biological|    2|
|Severe Storm(s)|    2|
|      Hurricane|    3|
+---------------+-----+

### Incident type by declaration year (count)

In [84]:
cleaned_disasters_df.groupby("incidentType", "declarationYear").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+---------------+-----+
|   incidentType|declarationYear|count|
+---------------+---------------+-----+
|Severe Storm(s)|           2012|    1|
|      Hurricane|           2017|    2|
|           Fire|           2017|    1|
|      Hurricane|           2019|    1|
|     Biological|           2020|    2|
|Severe Storm(s)|           2021|    1|
+---------------+---------------+-----+

### Incident type by declaration month (count)

In [85]:
cleaned_disasters_df.groupby("incidentType", "declarationMonth").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+----------------+-----+
|   incidentType|declarationMonth|count|
+---------------+----------------+-----+
|     Biological|              03|    2|
|Severe Storm(s)|              07|    2|
|      Hurricane|              08|    1|
|           Fire|              04|    1|
|      Hurricane|              09|    2|
+---------------+----------------+-----+

# Write Spark DF to S3 bucket as a CSV

In [196]:
# coalesce(1) writes csv to one file in S3
cleaned_disasters_df.coalesce(1) \
    .write.format('csv') \
    .mode('overwrite') \
    .option("header", "true") \
    .save(f's3://real-estate-filtered-data/county-disasters/{county}.csv/')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…